In [1]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 110%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [2]:
import numpy as np
import pandas as pd

from get_data.hh_requests import vacancy_keyword_check, update_vacancies_description
from text_processing.build_vacancy_text import build_vacancy_text

In [3]:
df = pd.DataFrame(columns=["id", "under consideration", "status", "description", "telegram moder",
                           "have full descript", "vacancy text",  "vacancy tedited", "update time", "logs"])
df = vacancy_keyword_check("Motion designer", df, period=5)
df = update_vacancies_description(df)
df.head(5)

# saving data backup TODO:add date and profession to file name
df[["description","snippet.requirement","snippet.responsibility"]].to_csv("description.csv")

/home/ambisinistra/miniconda3/envs/HHbot/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [4]:
df.head()

,id,under consideration,status,description,telegram moder,have full descript,vacancy text,vacancy tedited,update time,logs,...,address.metro.station_id,address.metro.line_id,address.metro.lat,address.metro.lng,department.id,department.name,employer.logo_urls,contacts.name,contacts.email,contacts.phones
0,38793634,0,unchecked,Arival is an early-stage digital banking start...,NaN,1,NaN,NaN,NaN,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,38708483,0,unchecked,"\n\nПривет!\n\nЕсли ты любишь моушн дизайн, ум...",NaN,1,NaN,NaN,NaN,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,38948196,0,unchecked,\n\nМы ищем человека в команду видео продакшен...,NaN,1,NaN,NaN,NaN,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,38990997,0,unchecked,"\n\nMotionads — креативная студия , специализи...",NaN,1,NaN,NaN,NaN,,...,15.216,15,59.956112,30.31889,NaN,NaN,NaN,NaN,NaN,NaN
0,38522226,0,unchecked,\n\nМы в **Wargaming** ******St.Petersburg** р...,NaN,1,NaN,NaN,NaN,,...,NaN,NaN,NaN,NaN,662769-662769-piter,"Wargaming, Saint Petersburg (Lesta Studio)",NaN,NaN,NaN,NaN


In [8]:
vacancy = build_vacancy_text(2, df)

print(vacancy)

Motion Designer
Crazy Panda
Москва

Что вы будете делать::
Участие в создании роликов по выпущенным проектам или проектам в разработке
2D и 3D анимация
Композинг и цветокоррекция
Эффекты

Что вы должны знать::Профессиональное владение одним из 3D редакторов: Maya, Blender, Cinema4D
Профессиональное владение: Photoshop, AfterEffects
Опыт работы с Illustrator, Premier, Substance Painter, Аudition будет плюсом
Умение рисовать от руки и работать с кривыми - большой плюс

Что для нас важно::Вы постоянно развиваете свой творческий кругозор и вкус как в плане софта, так и в плане потребляемого контента (игры, фильмы, лекции)
Чувствуете ритм, понимаете композицию, знает как правильно выбирать цвета
Любите процесс анимирования (оживления) и постоянно стараетесь улучшить свой скилл в этом деле
Видите для себя челенджи в казалось бы рутинных задачах и при необходимости создаете их самостоятельно

Что мы предлагаем::Достойная белая заработная плата;
Комфортное рабочее место со всем необходимым;
От

In [7]:
from my_telebot_utils import * #make_moder_keyboard, make_post_keyboard, make_suspicious_keyboard, make_irrelevant_keyboard, make_edit_keyboard
#from telebot.types import InlineKeyboardMarkup
#from telebot.types import InlineKeyboardButton 

from token import get_token
import telebot

TOKEN = get_token()
bot = telebot.TeleBot(token=TOKEN, threaded=False)
counter = 0

# When moderator ask for vacancy bot should check for unnocuppied one,
# mark it and give to moderator

@bot.message_handler(commands=["get"])
def check(message):
    keyboard = make_moder_keyboard()
    
    index = list(df[df["status"]=="unchecked"].index)[0]
    user_id = message.from_user.id
    
    text = build_vacancy_text(index, df)
    df["vacancy text"].iloc[index] = text
    df["under consideration"].iloc[index] = 1
    df["telegram moder"].iloc[index] = user_id
    df["status"].iloc[index] = "checking_by " + str(user_id)
    df["logs"].iloc[index] += "send for check to " + str(user_id) + "\n" #TODO: add date
    
    bot.send_message(message.chat.id, text=text, reply_markup=keyboard)
    #print (message.chat.id)

@bot.callback_query_handler(func=lambda call: True)
def callback_query(call):
    index = df[df["telegram moder"] == call.from_user.id].index #preassume there only one row
    #recieving what key on message keyboard was pressed and editing keyboard accordingly
    if call.data == "post":
        keyboard = make_post_keyboard()
        bot.send_message(text=call.message.text, chat_id=-1001267139997)
        df["status"].iloc[index] = "posted"
        df["telegram moder"].iloc[index] = -1
        df["logs"] += "posted to " + str(-1001267139997) + "\n"
        bot.answer_callback_query(call.id, "vacancy was posted")
    elif call.data == "irre":
        keyboard = make_irrelevant_keyboard()
        df["status"].iloc[index] = "irrelevant"
        df["telegram moder"].iloc[index] = -1
        df["logs"] += "marked as irrelevant \n"
        bot.answer_callback_query(call.id, "vacancy was marked as irrelevant")
    elif call.data == "susp":
        keyboard = make_suspicious_keyboard()
        df["status"].iloc[index] = "suspicious"
        df["telegram moder"].iloc[index] = -1
        df["logs"] += "marked as suspicious \n"
        bot.answer_callback_query(call.id, "vacancy was marked as suspicious")
    elif call.data == "othe":
        keyboard = make_irrelevant_keyboard()
        df["status"].iloc[index] = "other"
        df["telegram moder"].iloc[index] = -1
        df["logs"] += "marked as other \n"
        bot.answer_callback_query(call.id, "vacancy was marked as out of a box")
    elif call.data == "edit":
        #TODO
        keyboard = make_edit_keyboard()
        bot.answer_callback_query(call.id, "please send me editeded vacancy post")
    
    # depend on type of callback editing message keyboard. In case of edit button change text message also  
    bot.edit_message_text(text=call.message.text, chat_id = call.from_user.id, message_id=call.message.message_id, reply_markup=keyboard)
    
bot.polling()

ImportError: cannot import name 'get_token' from 'token' (/home/ambisinistra/miniconda3/envs/HHbot/lib/python3.8/token.py)